In [1]:
!pip install -U langchain langchain_experimental openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: openai
    Found existing installation: openai 2.16.0
    Uninstalling openai-2.16.0:
      Successfully uninstalled openai-2.16.0
  Attempting uninstall: langchain-core
    Found exis

In [2]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 3.0 MB/s eta 0:00:00


In [5]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from getpass import getpass
import time

# =======================
# 1. Schema
# =======================
class Proizvod(BaseModel):
    sifra_proizvod: str
    kategorija: str
    podkategorija: str
    brend: str
    celen_pol: str
    cenoven_opseg: str
    sezona: str
    status_dostapnost: str
    prosecna_ocenka: float
    broj_prodazbi: int

# =======================
# 2. Examples
# =======================
examples = [
    {
        "example": """
sifra_proizvod: P001, kategorija: Електроника, podkategorija: Паметни часовници, brend: Xiaomi, celen_pol: Унисекс, cenoven_opseg: Средна класа, sezona: Целогодишно, status_dostapnost: На залиха, prosecna_ocenka: 4.5, broj_prodazbi: 1240
"""
    },
    {
        "example": """
sifra_proizvod: P002, kategorija: Облека, podkategorija: Фустани, brend: Zara, celen_pol: Женски, cenoven_opseg: Премиум, sezona: Сезонски, status_dostapnost: Ограничено, prosecna_ocenka: 4.8, broj_prodazbi: 540
"""
    },
    {
        "example": """
sifra_proizvod: P003, kategorija: Храна, podkategorija: Чоколади, brend: Milka, celen_pol: Унисекс, cenoven_opseg: Буџет, sezona: Целогодишно, status_dostapnost: На залиха, prosecna_ocenka: 4.2, broj_prodazbi: 2300
"""
    },
    {
        "example": """
sifra_proizvod: P004, kategorija: Облека, podkategorija: Маици, brend: H&M, celen_pol: Машки, cenoven_opseg: Средна класа, sezona: Целогодишно, status_dostapnost: Нема залиха, prosecna_ocenka: 4.0, broj_prodazbi: 870
"""
    },
    {
        "example": """
sifra_proizvod: P005, kategorija: Електроника, podkategorija: Лаптопи, brend: Lenovo, celen_pol: Унисекс, cenoven_opseg: Премиум, sezona: Целогодишно, status_dostapnost: Ограничено, prosecna_ocenka: 4.7, broj_prodazbi: 320
"""
    }
]

# =======================
# 3. Example prompt
# =======================
example_prompt = PromptTemplate(
    input_variables=["example"],
    template="{example}"
)

# =======================
# 4. Few-shot prompt
# =======================
parser = PydanticOutputParser(pydantic_object=Proizvod)

few_shot_prompt = FewShotPromptTemplate(
    prefix="Подолу се дадени примери за записи за производи:\n",
    examples=examples,
    suffix="""
Генерирај НОВ реалистичен производ со ист формат.

ВАЖНО:
- НЕ повторувај иста категорија + подкатегорија како претходните примери, стави и нови категории
- Производот мора да биде логичен и реалистичен

{format_instructions}
""",
    input_variables=[],
    example_prompt=example_prompt,
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)

# =======================
# 5. OpenRouter LLM
# =======================
openrouter_key = getpass("Внеси OpenRouter API key: ")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key=openrouter_key,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.7
)

# =======================
# 6. Generate MULTIPLE products
# =======================
NUM_PRODUCTS = 100
results = []

for i in range(NUM_PRODUCTS):
    response = llm.invoke(
        few_shot_prompt.format() + f"\nID: {i}"
    )
    proizvod = parser.parse(response.content)

    proizvod.sifra_proizvod = f"P{i+1:04d}"

    results.append(proizvod)

    print(f"✅ Генериран {i+1}/{NUM_PRODUCTS}")
    time.sleep(1)

# =======================
# 7. Print results
# =======================

for p in results:
    print(p)

Внеси OpenRouter API key: ··········
✅ Генериран 1/100
✅ Генериран 2/100
✅ Генериран 3/100
✅ Генериран 4/100
✅ Генериран 5/100
✅ Генериран 6/100
✅ Генериран 7/100
✅ Генериран 8/100
✅ Генериран 9/100
✅ Генериран 10/100
✅ Генериран 11/100
✅ Генериран 12/100
✅ Генериран 13/100
✅ Генериран 14/100
✅ Генериран 15/100
✅ Генериран 16/100
✅ Генериран 17/100
✅ Генериран 18/100
✅ Генериран 19/100
✅ Генериран 20/100
✅ Генериран 21/100
✅ Генериран 22/100
✅ Генериран 23/100
✅ Генериран 24/100
✅ Генериран 25/100
✅ Генериран 26/100
✅ Генериран 27/100
✅ Генериран 28/100
✅ Генериран 29/100
✅ Генериран 30/100
✅ Генериран 31/100
✅ Генериран 32/100
✅ Генериран 33/100
✅ Генериран 34/100
✅ Генериран 35/100
✅ Генериран 36/100
✅ Генериран 37/100
✅ Генериран 38/100
✅ Генериран 39/100
✅ Генериран 40/100
✅ Генериран 41/100
✅ Генериран 42/100
✅ Генериран 43/100
✅ Генериран 44/100
✅ Генериран 45/100
✅ Генериран 46/100
✅ Генериран 47/100
✅ Генериран 48/100
✅ Генериран 49/100
✅ Генериран 50/100
✅ Генериран 51/100
✅ Г

In [6]:
import json

data = [p.dict() for p in results]
path = "/content/drive/MyDrive/Vezilka/LangChain & GPT-4o/products_results.json"


with open(path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✅ Saved to products_results.json")

✅ Saved to products_results.json


/tmp/ipython-input-741912887.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  data = [p.dict() for p in results]
